In [110]:
import pandas as pd

df = pd.read_csv("관광데이터.csv", encoding="cp949")

In [111]:
df

,Unnamed: 0,ID,year,month,quarter,country,gender,age,visit_purpose,num_visits,...,shopping_percent,food,landscape,heritage,language,safety,budget,accommodation,transport,navigation
0,0,57.0,2024,1,1,12,2,5,1,4,...,0.184502,NaN,5.0,NaN,5.0,5.0,5.0,5.0,5.0,5.0
1,1,58.0,2024,1,1,3,2,4,1,4,...,0.356062,5.0,NaN,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2,2,59.0,2024,1,1,3,2,3,1,3,...,0.302984,5.0,NaN,NaN,3.0,5.0,5.0,5.0,5.0,3.0
3,3,60.0,2024,1,1,1,1,2,1,1,...,0.151286,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
4,4,61.0,2024,1,1,3,2,2,1,4,...,0.588120,NaN,NaN,4.0,3.0,4.0,4.0,4.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32059,64932,9002.0,2018,8,3,2,1,5,1,4,...,0.208333,3.0,4.0,4.0,4.0,3.0,4.0,3.0,4.0,4.0
32060,64936,9002.0,2018,12,4,2,2,2,1,1,...,0.205128,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
32061,64941,9003.0,2018,6,2,2,1,5,1,4,...,0.257143,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0
32062,64945,9003.0,2018,10,4,2,2,5,1,3,...,0.191489,5.0,4.0,4.0,3.0,4.0,3.0,4.0,4.0,4.0


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32064 entries, 0 to 32063
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             32064 non-null  int64  
 1   ID                     32064 non-null  float64
 2   year                   32064 non-null  int64  
 3   month                  32064 non-null  int64  
 4   quarter                32064 non-null  int64  
 5   country                32064 non-null  int64  
 6   gender                 32064 non-null  int64  
 7   age                    32064 non-null  int64  
 8   visit_purpose          32064 non-null  int64  
 9   num_visits             32064 non-null  int64  
 10  revisit_indicator      32064 non-null  int64  
 11  travel_type            32064 non-null  int64  
 12  satisfaction           32064 non-null  int64  
 13  revisit_intent         32064 non-null  int64  
 14  recommend_intent       32064 non-null  int64  
 15  vi

대략적인 전략

사용 데이터: 외래관광설문조사

1. 지도 학습 기반의 '의미 있는' 여행자 군집화
    - 후보 군집
        * 필수: 고려요인, 국적
        * 추가 대상: 성별, 나이
    - 최적 군집 변수 조합 탐색
        * k-means
        * 실루엣 점수
    - 최종 군집 형성 및 프로파일링
        * 각 군집의 인구통계, 여행 행태, 소비 패턴 등을 분석하여 "MZ세대 K-컬처 팬", "고소비 럭셔리 쇼핑객", "장기 체류 미식 탐방가" 등 구체적인 이름과 특성을 부여

2. 개인화된 여행지 추천 모델링 및 원인 분석
    - 개인화 추천 모델 훈련
        * 1에서 생성된 군집 라벨과 방문 지역(원-핫 인코딩)을 포함한 전체 변수를 사용하여, 만족도를 다시 예측하는 고도화된 2차 모델(XGBoost 등)을 훈련
    - 시뮬레이션을 통한 최적의 '서울+지방' 조합 탐색
        * 각 군집별 만족도를 극대화하는 추천 여행지 도출
        * 각 군집별로 가장 높은 예측 만족도 점수를 보인 최적의 (군집, 추천 지방) 조합 리스트
    - SHAP를 이용한 추천 근거 분석
        * "MZ세대 K-컬처 팬이 부산을 방문했을 때 만족도가 높은 이유는
            , 이들의 긴 체류 기간 특성이 부산의 다양한 K-콘텐츠 명소를 충분히 즐기게 하고
            , 식도락 활동에 대한 높은 관심이 부산의 음식 문화와 시너지를 내기 때문이다." 와 같은 구체적인 해석.

3. LLM 파인튜닝을 위한 데이터화 및 적용
    - Instruction 데이터셋 구축
    - LLM 파인튜닝 및 서비스화

방문객 고려요인 변수로 추가 -> 주요시장으로 나눔
2018: Q5_2A1
2019: Q5_2A1
2023: Q3_1a1
2024: Q3_1a1

In [113]:
# 2018
{
  "Q5_2A1_to_Q5_2A3": {
    "description": "문5-2. 한국 방문 선택 시 고려 요인 1~3",
    "mapping": {
      "1": "음식/미식 탐방",
      "2": "쇼핑",
      "3": "자연 풍경 감상",
      "4": "친구, 친지 방문", # 2019 X
      "5": "역사/문화 유적 탐방", # 2019: "4"
      "6": "템플스테이, 태권도 등 한국 전통 문화 체험", # 2019 X
      "7": "유흥/놀이 시설", 
      "8": "패션, 유행 등 세련된 현대 문화", # 2019: "5"
      "9": "K-POP/한류스타 관련 팬 미팅, 촬영지 등 경험", # 2019: "6"
      "10": "치료 및 미용 서비스", # 2019: "8"
      "11": "레저 및 스포츠 활동", # 2019: "9"
      "12": "자국과의 이동 거리", # 2019: "11"
      "13": "경제적인 여행비용", # 2019: "12"
      "14": "휴가 시간", # 2019: "10"
      "15": "기후/뚜렷한 사계절", # 2019 X
      "16": "숙박/교통 등 관광 편의성 및 안전성", # 2019 X
      "17": "기타"
    }
  }
}

# 2019
{
  "Q5_2A1_to_Q5_2A3": {
    "description": "문5-2. 한국 방문 선택 시 고려 요인 (13개 항목 버전)",
    "mapping": {
      "1": "음식/미식 탐방",
      "2": "쇼핑",
      "3": "자연 풍경 감상",
      "4": "역사/문화 유적/전통문화 체험",
      "5": "패션, 유행 등 세련된 현대 문화 체험",
      "6": "K-POP/한류스타 관련 팬 미팅, 촬영지 등 경험",
      "7": "유흥/놀이 시설 체험",
      "8": "치료 및 미용 서비스 체험",
      "9": "레저 및 스포츠 활동",
      "10": "휴가 시간",
      "11": "자국과의 이동 거리",
      "12": "경제적인 여행비용",
      "13": "기타"
    }
  }
}

# 2023
{
  "Q3_1a1_to_Q3_1a3_v_16items": {
    "description": "문3-1. 고려한 관광 활동 (16개 항목 버전)",
    "mapping": {
      "1": "식도락(음식/미식) 관광",
      "2": "쇼핑",
      "3": "자연경관 감상",
      "4": "고궁/역사 유적지 방문",
      "5": "전통문화체험 (한복체험, 태권도, 한국 음식 만들기 등)",
      "6": "박물관, 전시관 관람",
      "7": "K-POP/한류스타 관련 공연장 및 드라마/영화 촬영지 방문",
      "8": "연극, 뮤지컬, 발레 등 공연 관람",
      "9": "지역 축제 참여",
      "10": "유흥(나이트라이프 활동 및 카지노)",
      "11": "오락(놀이공원 및 테마파크 방문 등)",
      "12": "뷰티/미용 관광(마사지샵, 헤어샵, 네일케어 등 방문)",
      "13": "치료/건강검진(피부과, 성형외과, 내외과 입원, 수술 등)",
      "14": "스포츠/레포츠 관람 (야구, 축구, 농구 등)",
      "15": "스포츠/레포츠 참가 (스키, 수영, 골프, 캠핑, 등반, 자전거, 래프팅 등)",
      "16": "기타"
    }
  }
}

# 2024
{
  "Q3_1a1_to_Q3_1a3_v_17items": {
    "description": "문3-1. 고려한 관광 활동 (17개 항목 버전)",
    "mapping": {
      "1": "식도락(음식/미식) 관광",
      "2": "쇼핑",
      "3": "자연경관 감상",
      "4": "휴양/휴식(웰니스)",
      "5": "고궁/역사 유적지 방문",
      "6": "전통문화체험 (한복체험, 태권도, 한국 음식 만들기 등)",
      "7": "박물관, 전시관 관람",
      "8": "K-POP/한류스타 관련 공연장 및 드라마/영화 촬영지 방문",
      "9": "연극, 뮤지컬, 발레 등 공연 관람",
      "10": "지역 축제 참여",
      "11": "유흥(나이트라이프 활동 및 카지노)",
      "12": "오락(놀이공원 및 테마파크 방문 등)",
      "13": "뷰티/미용 관광(마사지샵, 헤어샵, 네일케어 등 방문)",
      "14": "치료/건강검진(피부과, 성형외과, 내외과 입원, 수술 등)",
      "15": "스포츠/레포츠 관람 (야구, 축구, 농구 등)",
      "16": "스포츠/레포츠 참가 (스키, 수영, 골프, 캠핑, 등반, 자전거, 래프팅 등)",
      "17": "기타"
    }
  }
}


{'Q3_1a1_to_Q3_1a3_v_17items': {'description': '문3-1. 고려한 관광 활동 (17개 항목 버전)',
  'mapping': {'1': '식도락(음식/미식) 관광',
   '2': '쇼핑',
   '3': '자연경관 감상',
   '4': '휴양/휴식(웰니스)',
   '5': '고궁/역사 유적지 방문',
   '6': '전통문화체험 (한복체험, 태권도, 한국 음식 만들기 등)',
   '7': '박물관, 전시관 관람',
   '8': 'K-POP/한류스타 관련 공연장 및 드라마/영화 촬영지 방문',
   '9': '연극, 뮤지컬, 발레 등 공연 관람',
   '10': '지역 축제 참여',
   '11': '유흥(나이트라이프 활동 및 카지노)',
   '12': '오락(놀이공원 및 테마파크 방문 등)',
   '13': '뷰티/미용 관광(마사지샵, 헤어샵, 네일케어 등 방문)',
   '14': '치료/건강검진(피부과, 성형외과, 내외과 입원, 수술 등)',
   '15': '스포츠/레포츠 관람 (야구, 축구, 농구 등)',
   '16': '스포츠/레포츠 참가 (스키, 수영, 골프, 캠핑, 등반, 자전거, 래프팅 등)',
   '17': '기타'}}}

in_2024
    {
      "1": "식도락(음식/미식) 관광",
      "2": "쇼핑",
      "3": "자연경관 감상",
      "4": "휴양/휴식(웰니스)", # 2023: X
      "5": "고궁/역사 유적지 방문",
      "6": "전통문화체험 (한복체험, 태권도, 한국 음식 만들기 등)",
      "7": "박물관, 전시관 관람",
      "8": "K-POP/한류스타 관련 공연장 및 드라마/영화 촬영지 방문",
      "9": "연극, 뮤지컬, 발레 등 공연 관람",
      "10": "지역 축제 참여",
      "11": "유흥(나이트라이프 활동 및 카지노)",
      "12": "오락(놀이공원 및 테마파크 방문 등)",
      "13": "뷰티/미용 관광(마사지샵, 헤어샵, 네일케어 등 방문)",
      "14": "치료/건강검진(피부과, 성형외과, 내외과 입원, 수술 등)",
      "15": "스포츠/레포츠 관람 (야구, 축구, 농구 등)",
      "16": "스포츠/레포츠 참가 (스키, 수영, 골프, 캠핑, 등반, 자전거, 래프팅 등)",
      "17": "기타"
    }

in_2018
  {
    "description": "문5-2. 한국 방문 선택 시 고려 요인 (13개 항목 버전)",
    "mapping": {
      "1": "음식/미식 탐방",
      "2": "쇼핑",
      "3": "자연 풍경 감상",
      "4": "친구, 친지 방문", # 2019 X
      "5": "역사/문화 유적 탐방", # 2019: "4"
      "6": "템플스테이, 태권도 등 한국 전통 문화 체험", # 2019 X
      "7": "유흥/놀이 시설", 
      "8": "패션, 유행 등 세련된 현대 문화", # 2019: "5"
      "9": "K-POP/한류스타 관련 팬 미팅, 촬영지 등 경험", # 2019: "6"
      "10": "치료 및 미용 서비스", # 2019: "8"
      "11": "레저 및 스포츠 활동", # 2019: "9"
      "12": "자국과의 이동 거리", # 2019: "11"
      "13": "경제적인 여행비용", # 2019: "12"
      "14": "휴가 시간", # 2019: "10"
      "15": "기후/뚜렷한 사계절", # 2019 X
      "16": "숙박/교통 등 관광 편의성 및 안전성", # 2019 X
      "17": "기타"
    }
  }
  

최종
{
    "1": "식도락(음식/미식) 관광", 2023: 1, 2019: 1, 2018: 1
    "2": "쇼핑", 2023: 2, 2019: 2, 2018: 2 
    "3": "자연경관 감상", 2023: 3, 2019: 3, 2018: 3
    "4": "휴양/휴식(웰니스)",  2023: X, 2019: 9, 2018: 10
    "5": "고궁/역사 유적지 방문", 2023: 4, 2019: 4, 2018: 5
    "6": "전통문화체험 (한복체험, 태권도, 한국 음식 만들기 등)", 2023: 5, 2019: X, 2018: 6
    "7": "박물관, 전시관 관람",  2023: 6, 2019: X, 2018: X
    "8": "K-POP/한류스타 관련 공연장 및 드라마/영화 촬영지 방문",  2023: 7, 2019: 6, 2018: 9
    "9": "연극, 뮤지컬, 발레 등 공연 관람",  2023: 8, 2019: X, 2018: X
    "10": "지역 축제 참여",  2023: 9, 2019: X, 2018: X
    "11": "유흥(나이트라이프 활동 및 카지노)",  2023: 10, 2019: 7, 2018: 7
    "12": "오락(놀이공원 및 테마파크 방문 등)",  2023: 11, 2019: 7, 2018: 7
    "13": "뷰티/미용 관광(마사지샵, 헤어샵, 네일케어 등 방문)",  2023: 12, 2019: 8, 2018: 10
    "14": "치료/건강검진(피부과, 성형외과, 내외과 입원, 수술 등)",  2023: 13, 2019: 8, 2018: 10
    "15": "스포츠/레포츠 관람 (야구, 축구, 농구 등)",  2023: 14, 2019: 9, 2018: 11 
    "16": "스포츠/레포츠 참가 (스키, 수영, 골프, 캠핑, 등반, 자전거, 래프팅 등)",  2023: 15, 2019: 9, 2018: 11
    "17": "기타"  2023: 16, 2019: 13, 2018: 17
}

In [114]:
processed_df = df.copy()

cols_to_process = ['food_percent', 'shopping_percent', 'accommodation_percent'
                  , 'food', 'landscape', 'heritage', 'language', 'safety', 'budget'
                  , 'accommodation', 'transport', 'navigation']
# for 문을 사용해 각 컬럼에 순차적으로 로직 적용
for col in cols_to_process:
    processed_df[col].fillna(0, inplace=True)
    
    print(f"\n'{col}' 컬럼 처리 완료. (대체값: {0})")

cols_to_compare = ['accommodation_percent', 'food_percent', 'shopping_percent']

max_col_names = processed_df[cols_to_compare].idxmax(axis=1)

value_map = {
    'accommodation_percent': 1,
    'food_percent': 2,
    'shopping_percent': 3
}

processed_df['main_expense'] = max_col_names.map(value_map)
#processed_df.drop(columns= cols_to_compare, inplace = True)

convert_dict_int = {
    'food': 'int',
    'landscape': 'int',
    'heritage': 'int',
    'main_expense': 'int',
    'language': 'int',
    'safety': 'int',
    'budget': 'int',
    'accommodation': 'int',
    'transport': 'int',
    'navigation': 'int',
}

processed_df = processed_df.astype(convert_dict_int)


convert_dict_wo_order = {
    'country': 'category',
    'gender': 'category',
    'revisit_indicator': 'category',
    'visit_local_indicator': 'category',
    'seoul': 'category',
    'gyeonggi': 'category',
    'incheon': 'category',
    'gangwon': 'category',
    'chungcheong': 'category',
    'gyeongsang': 'category',
    'jeolla': 'category',
    'jeju': 'category',
    'food': 'category',
    'landscape': 'category',
    'heritage': 'category',
    'main_expense': 'category',
    'language': 'category',
    'safety': 'category',
    'budget': 'category',
    'accommodation': 'category',
    'transport': 'category',
    'navigation': 'category',
    'month': 'category'
}

processed_df = processed_df.astype(convert_dict_wo_order)

from pandas.api.types import CategoricalDtype

convert_dict_w_order = {
    'age': CategoricalDtype(categories=[1, 2, 3, 4, 5, 6], ordered=True),
    #'satisfaction': CategoricalDtype(categories=[1, 2, 3, 4, 5], ordered=True),
    'revisit_intent': CategoricalDtype(categories=[1, 2, 3, 4, 5], ordered=True),
    'recommend_intent': CategoricalDtype(categories=[1, 2, 3, 4, 5], ordered=True),
    

}


'food_percent' 컬럼 처리 완료. (대체값: 0)

'shopping_percent' 컬럼 처리 완료. (대체값: 0)

'accommodation_percent' 컬럼 처리 완료. (대체값: 0)

'food' 컬럼 처리 완료. (대체값: 0)

'landscape' 컬럼 처리 완료. (대체값: 0)

'heritage' 컬럼 처리 완료. (대체값: 0)

'language' 컬럼 처리 완료. (대체값: 0)

'safety' 컬럼 처리 완료. (대체값: 0)

'budget' 컬럼 처리 완료. (대체값: 0)

'accommodation' 컬럼 처리 완료. (대체값: 0)

'transport' 컬럼 처리 완료. (대체값: 0)

'navigation' 컬럼 처리 완료. (대체값: 0)


C:\Users\SyngyeonTak\AppData\Local\Temp\ipykernel_86772\1332278453.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  processed_df[col].fillna(0, inplace=True)


In [115]:
processed_df.drop(columns= cols_to_compare, inplace = True)


In [116]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32064 entries, 0 to 32063
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Unnamed: 0             32064 non-null  int64   
 1   ID                     32064 non-null  float64 
 2   year                   32064 non-null  int64   
 3   month                  32064 non-null  category
 4   quarter                32064 non-null  int64   
 5   country                32064 non-null  category
 6   gender                 32064 non-null  category
 7   age                    32064 non-null  int64   
 8   visit_purpose          32064 non-null  int64   
 9   num_visits             32064 non-null  int64   
 10  revisit_indicator      32064 non-null  category
 11  travel_type            32064 non-null  int64   
 12  satisfaction           32064 non-null  int64   
 13  revisit_intent         32064 non-null  int64   
 14  recommend_intent       32064 non-null 

In [117]:
nan_counts = processed_df.isnull().sum()

# NaN 개수가 0보다 큰 컬럼들만 필터링
columns_with_nan = nan_counts[nan_counts > 0]

print("--- NaN을 포함한 변수와 그 개수 ---")
print(columns_with_nan)
print(processed_df.columns)

--- NaN을 포함한 변수와 그 개수 ---
Series([], dtype: int64)
Index(['Unnamed: 0', 'ID', 'year', 'month', 'quarter', 'country', 'gender',
       'age', 'visit_purpose', 'num_visits', 'revisit_indicator',
       'travel_type', 'satisfaction', 'revisit_intent', 'recommend_intent',
       'visit_local_indicator', 'seoul', 'gyeonggi', 'incheon', 'gangwon',
       'chungcheong', 'gyeongsang', 'jeolla', 'jeju', 'stay_duration',
       'total_duration', 'total_expense', 'accommodation_cost', 'food_cost',
       'shopping_cost', 'food', 'landscape', 'heritage', 'language', 'safety',
       'budget', 'accommodation', 'transport', 'navigation', 'main_expense'],
      dtype='object')


In [118]:
import math

print(processed_df.columns)
columns_to_drop = ["Unnamed: 0", "ID", "year", "quarter", "visit_purpose"
    , "num_visits", "travel_type", "stay_duration"
    , "total_expense", "accommodation_cost", "food_cost", "shopping_cost"
    ]

processed_df.drop(columns= columns_to_drop, inplace = True)


cols_to_onehot = ['food', 'landscape', 'heritage', 'language', 'safety', 'budget'
                  , 'accommodation', 'transport', 'navigation']
# 3. 원-핫 인코딩할 데이터와 유지할 데이터를 분리
df_to_encode = processed_df[cols_to_onehot]
df_to_keep = processed_df.drop(columns=cols_to_onehot)
print(df_to_encode.columns.tolist())
# 4. 원-핫 인코딩 실행
df_encoded = pd.get_dummies(
    df_to_encode, 
    prefix=cols_to_onehot,
    prefix_sep='_',
    dummy_na=True,
    dtype=int
)

processed_df = pd.concat([df_to_keep, df_encoded], axis=1)


Index(['Unnamed: 0', 'ID', 'year', 'month', 'quarter', 'country', 'gender',
       'age', 'visit_purpose', 'num_visits', 'revisit_indicator',
       'travel_type', 'satisfaction', 'revisit_intent', 'recommend_intent',
       'visit_local_indicator', 'seoul', 'gyeonggi', 'incheon', 'gangwon',
       'chungcheong', 'gyeongsang', 'jeolla', 'jeju', 'stay_duration',
       'total_duration', 'total_expense', 'accommodation_cost', 'food_cost',
       'shopping_cost', 'food', 'landscape', 'heritage', 'language', 'safety',
       'budget', 'accommodation', 'transport', 'navigation', 'main_expense'],
      dtype='object')
['food', 'landscape', 'heritage', 'language', 'safety', 'budget', 'accommodation', 'transport', 'navigation']


In [119]:
processed_df.drop(columns=['food_nan', 'landscape_nan', 'heritage_nan', 'language_nan', 'safety_nan', 'budget_nan', 'accommodation_nan', 'transport_nan', 'navigation_nan'], inplace= True)

In [120]:
nan_counts = processed_df.isnull().sum()

# NaN 개수가 0보다 큰 컬럼들만 필터링
columns_with_nan = nan_counts[nan_counts > 0]

print("--- NaN을 포함한 변수와 그 개수 ---")
print(columns_with_nan)

--- NaN을 포함한 변수와 그 개수 ---
Series([], dtype: int64)


In [121]:
processed_df.columns

Index(['month', 'country', 'gender', 'age', 'revisit_indicator',
       'satisfaction', 'revisit_intent', 'recommend_intent',
       'visit_local_indicator', 'seoul', 'gyeonggi', 'incheon', 'gangwon',
       'chungcheong', 'gyeongsang', 'jeolla', 'jeju', 'total_duration',
       'main_expense', 'food_0', 'food_1', 'food_2', 'food_3', 'food_4',
       'food_5', 'landscape_0', 'landscape_1', 'landscape_2', 'landscape_3',
       'landscape_4', 'landscape_5', 'heritage_0', 'heritage_1', 'heritage_2',
       'heritage_3', 'heritage_4', 'heritage_5', 'language_0', 'language_1',
       'language_2', 'language_3', 'language_4', 'language_5', 'safety_0',
       'safety_1', 'safety_2', 'safety_3', 'safety_4', 'safety_5', 'budget_0',
       'budget_1', 'budget_2', 'budget_3', 'budget_4', 'budget_5',
       'accommodation_0', 'accommodation_1', 'accommodation_2',
       'accommodation_3', 'accommodation_4', 'accommodation_5', 'transport_0',
       'transport_1', 'transport_2', 'transport_3', 'tran

In [122]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32064 entries, 0 to 32063
Data columns (total 73 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   month                  32064 non-null  category
 1   country                32064 non-null  category
 2   gender                 32064 non-null  category
 3   age                    32064 non-null  int64   
 4   revisit_indicator      32064 non-null  category
 5   satisfaction           32064 non-null  int64   
 6   revisit_intent         32064 non-null  int64   
 7   recommend_intent       32064 non-null  int64   
 8   visit_local_indicator  32064 non-null  category
 9   seoul                  32064 non-null  category
 10  gyeonggi               32064 non-null  category
 11  incheon                32064 non-null  category
 12  gangwon                32064 non-null  category
 13  chungcheong            32064 non-null  category
 14  gyeongsang             32064 non-null 

In [123]:
processed_df

,month,country,gender,age,revisit_indicator,satisfaction,revisit_intent,recommend_intent,visit_local_indicator,seoul,...,transport_2,transport_3,transport_4,transport_5,navigation_0,navigation_1,navigation_2,navigation_3,navigation_4,navigation_5
0,1,12,2,5,1,5,5,5,0,1,...,0,0,0,1,0,0,0,0,0,1
1,1,3,2,4,1,5,5,5,0,1,...,0,0,1,0,0,0,0,0,1,0
2,1,3,2,3,1,4,4,4,0,1,...,0,0,0,1,0,0,0,1,0,0
3,1,1,1,2,0,5,5,5,0,1,...,0,0,0,1,0,0,0,0,0,1
4,1,3,2,2,1,4,5,5,1,1,...,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32059,8,2,1,5,1,4,5,5,2,0,...,0,0,1,0,0,0,0,0,1,0
32060,12,2,2,2,0,5,5,5,2,0,...,0,0,0,1,0,0,0,0,0,1
32061,6,2,1,5,1,4,4,4,2,0,...,0,0,1,0,0,0,0,0,1,0
32062,10,2,2,5,1,4,5,5,2,0,...,0,0,1,0,0,0,0,0,1,0


In [124]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# 방법 1: 엘보우 방법 (Elbow Method)
inertia = []
K_range = range(2, 11) # K를 2부터 10까지 테스트

for k in K_range:
    kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
    kmeans.fit(processed_df)
    inertia.append(kmeans.inertia_)

# 엘보우 그래프 시각화
plt.figure(figsize=(10, 5))
plt.plot(K_range, inertia, marker='o')
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Inertia')
plt.xticks(K_range)
plt.grid(True)
plt.show()

ImportError: cannot import name 'BackendFilter' from 'matplotlib.backends' (unknown location)

우선 위의 방식으로 군집을 나누어보고 군집마다 변수 특징이 있는지 본다.
만약 이렇다할 게 없으면 그냥 휴리스틱하게 군집을 나눌 수 밖에 없다.
주요활동으로 나누고 이들 안에서 군집을 나눈다. 
군집을 나눌때 "어떤 그룹, 어떤 그룹으로 나누고 이들에 가장 영향을 주는 요인은 이것으로 확인되었다." 라고 도출해버리자

그리고 이들 중에서 서울, 서울 + 지방으로 나눈 그룹을 나눈다.
그래서 해당 군집에서 어떤 지방을 갈때 어떤 만족도가 많이 오르는지 확인한다.
그리고 그 이유를 다른 설명 변수로 설명하게끔한다.

어쩃든 제일 관심있는건

"이 군집은 이러한 특징이 있다. 어느 지방으로 갔을 떄 가장 많이 오른 만족도 요인은 푸드/transport 등이다
그 요인은 다른 군집보다 높은 ** 변수에 있다." 라는 insight를 도출한다.

군집은 [고려요인, 국적, 연령, 성별] 정도로 나눈다

Step

250715
1. 고려요인 항목 필요
2. 군집화
2-1. 그냥 KMeans로 하였을 때 군집을 잘 나눈 변수를 확인해보기
2-2. 휴리스틱하게 군집을 나누기 [고려요인, 국적, 연령, 성별], 군집별 비중이 적당한지...

250716
3. 군집별 특징, 선호지방, 지방 여행 시 향상된 만족도 를 제시한다.
3-1. 변수별 평균 정보 제시
- 순서형 카테고리 혹은 수치형 변수는 평균
- 그냥 카테고리는 비중
- 원핫인코딩도 변수끼리의 비중

3-2. 서울 vs 서울 + 지방
- 이들은 어떤 특징이 있는지 체제기간, 쇼핑 비중, 국적 군집 퍼센트 (w.r.t. rest 군집 평균)
- 어느 지방을 많이 갔는지
- 어느 지방을 가면 어떤 선호도가 떨어지는지
- 어느 지방을 가면 어떤 선호도가 올라갔는지

특징: 체제기간, 



